# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,marka,1.7159,44.7717,299.86,81,93,3.49,SO,1698608884
1,1,isafjordur,66.0755,-23.1240,274.27,80,36,1.52,IS,1698608884
2,2,blackmans bay,-43.0167,147.3167,285.66,63,98,0.89,AU,1698608885
3,3,klyuchi,52.2667,79.1667,272.30,93,100,2.05,RU,1698608886
4,4,ust-nera,64.5667,143.2000,255.23,98,61,0.83,RU,1698608886


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humid_map = city_data_df.hvplot.points('Lng','Lat',geo=True, tiles = 'OSM', 
    frame_width = 700, frame_height = 500, color='City',
    alpha= 0.5, size='Humidity', scale= 0.8)

# Display the map
humid_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df['Max Temp']<=302.594)\
                               &(city_data_df['Max Temp']>=288.706)\
                               &(city_data_df['Cloudiness']>50)\
                               &(city_data_df['Humidity']<80)\
                               &(city_data_df['Wind Speed']>=2),:]

# Drop any rows with null values
vacation_df.dropna()

# Display sample data
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,xai-xai,-25.0519,33.6442,292.05,75,100,9.88,MZ,1698608897
44,44,atafu village,-8.5421,-172.5159,302.12,70,94,4.69,TK,1698608913
62,62,azul,-36.7770,-59.8585,290.66,62,100,2.23,AR,1698608925
66,66,andalgala,-27.5819,-66.3165,301.44,14,99,3.50,AR,1698608928
78,78,ad dawadimi,24.5077,44.3924,297.79,34,98,4.88,SA,1698608937


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
18,xai-xai,MZ,-25.0519,33.6442,75,
44,atafu village,TK,-8.5421,-172.5159,70,
62,azul,AR,-36.7770,-59.8585,62,
66,andalgala,AR,-27.5819,-66.3165,14,
78,ad dawadimi,SA,24.5077,44.3924,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
xai-xai - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
azul - nearest hotel: Gran Hotel Azul
andalgala - nearest hotel: Residencial Ayucaba
ad dawadimi - nearest hotel: No hotel found
mili - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lihue - nearest hotel: Kauai Palms
touros - nearest hotel: Pousada Atlântico
kaduna - nearest hotel: Assa Pyramid Hotel
caleta de carquin - nearest hotel: Hotel Mi Choacan
alofi - nearest hotel: Matavai Resort
west island - nearest hotel: Cocos Village Bungalows
sur - nearest hotel: Sur Hotel
mana - nearest hotel: Le Samana
bouarfa - nearest hotel: Climat du Maroc مناخ المغرب
jabor - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
cova figueira - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
inyonga - nearest hotel: Mikumi Lodge
nchelenge - nearest hotel: No hotel found
carutapera - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
18,xai-xai,MZ,-25.0519,33.6442,75,No hotel found
44,atafu village,TK,-8.5421,-172.5159,70,No hotel found
62,azul,AR,-36.7770,-59.8585,62,Gran Hotel Azul
66,andalgala,AR,-27.5819,-66.3165,14,Residencial Ayucaba
78,ad dawadimi,SA,24.5077,44.3924,34,No hotel found
88,mili,MH,6.0815,171.7350,78,No hotel found
90,wailua homesteads,US,22.0669,-159.3780,76,Hilton Garden Inn Kauai Wailua Bay
93,lihue,US,21.9789,-159.3672,75,Kauai Palms
97,touros,BR,-5.1989,-35.4608,75,Pousada Atlântico
100,kaduna,NG,10.5222,7.4383,28,Assa Pyramid Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng','Lat', geo=True, 
            tiles = 'OSM',frame_width = 700, frame_height = 500, 
            color='City',alpha=0.6, hover=True, hover_cols='all',
            use_index=False, size='Humidity',scale= 1.2)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)